In [0]:
# %sql
# CREATE OR REPLACE TABLE gold_layer_ticker_stream
# AS SELECT * FROM delta.`/Volumes/workspace/default/storage/gold/ticker_data_v7`;

In [0]:
# %sql
# CREATE OR REPLACE TABLE serving_layer_ticker_stream
# AS SELECT * FROM delta.`/Volumes/workspace/default/storage/serving/valuation_dashboard_v4`;

In [0]:
# %sql
# select * from serving_layer_ticker_stream
# where report_date < to_date('2025-09-01')

In [0]:
%sql
-- 1. Define the Security Logic Function
CREATE OR REPLACE FUNCTION filter_gold_access(report_date DATE)
RETURN 
  CASE 
    -- Scenario 1: Executives & Owners see EVERYTHING (TRUE)
    WHEN is_account_group_member('executives') OR is_account_group_member('data_owners') THEN TRUE
    
    -- Scenario 2: Analysts only see "Verified" data (High Sample Size)
    WHEN is_account_group_member('analysts') THEN report_date >= to_date('2025-10-01')
    
    -- Scenario 3: Everyone else sees NOTHING
    ELSE FALSE
  END;

In [0]:
%sql

-- 1. Create the Secure View directly on the Volume Path
CREATE OR REPLACE VIEW secure_gold_analytics AS
SELECT 
    -- [Column Level Security]: Mask 'cost_of_equity' for Analysts
    symbol,
    valuation_date,
    close_price,
    wacc,
    CASE 
        WHEN is_account_group_member('analysts') THEN NULL  -- Hide for Analysts
        ELSE cost_of_equity                                 -- Show for Execs/Owners
    END as cost_of_equity,
    
    momentum 
FROM 
    delta.`/Volumes/workspace/default/storage/serving/valuation_dashboard_v4`
;

-- 2. Documentation
COMMENT ON VIEW secure_gold_analytics IS 'Security View on Gold Volume. RLS: Sample Size > 20. CLS: Cost of Equity masked.';

In [0]:
%sql

CREATE OR REPLACE VIEW secure_gold_analytics AS
SELECT 
    -- [Column Level Security]: Mask 'cost_of_equity' for Analysts
    symbol,
    valuation_date,
    close_price,
    wacc,
    CASE 
        WHEN is_account_group_member('analysts') THEN NULL  -- Hide for Analysts
        ELSE cost_of_equity                                 -- Show for Execs/Owners
    END as cost_of_equity,
    
    momentum 
FROM 
    delta.`/Volumes/workspace/default/storage/serving/valuation_dashboard_v4`
WHERE 
  -- This effectively "Sets" the Row Filter
  filter_gold_access(report_date) = TRUE;

In [0]:
%sql


-- 2. Grant access ONLY to the View
GRANT SELECT ON VIEW secure_gold_analytics TO `analysts`;
GRANT SELECT ON VIEW secure_gold_analytics TO `executives`;

Executives: Run a query as an executive.

Result: They can see ALL rows (Row Filter returns TRUE) and ALL columns (Mask returns value).

Data Owners: Run OPTIMIZE gold_ticker_analytics as a data owner.

Result: It succeeds (because they have MODIFY/ALL).

Analysts: Run OPTIMIZE ... as an analyst.

Result: Access Denied (They only have SELECT).

In [0]:
%sql
select cost_of_equity from secure_gold_analytics

In [0]:
%sql
SELECT event_time, user_identity.email, service_name, action_name, response.error_message
FROM system.access.audit

ORDER BY event_time DESC;